# Python for Learning AI Week 2: Port Communication

Welcome to Week 2's exploration of Port Communication in Python! This notebook will teach you how to build networked applications by understanding and working with ports. Whether you're building client-server applications, microservices, or debugging network issues, understanding port communication is essential.

## What You'll Learn
- Understanding ports and network communication
- Creating TCP servers and clients
- Building HTTP servers
- Handling multiple connections
- Best practices for network programming

## Prerequisites
- Basic Python knowledge (covered in Week 1)
- Understanding of functions and error handling
- Familiarity with object-oriented programming

Let's start by installing the required packages:

In [ ]:
# Check if required packages are installed
try:
    import socket
    import threading
    import time
    import psutil
    print("✓ All required packages are available!")
except ImportError as e:
    print(f"✗ Missing package: {str(e)}")
    print("\nIMPORTANT: To install packages, run this command in your terminal (not in the notebook):")
    print("uv pip install psutil")
    print("\nWhy not install in the notebook?")
    print("- Installing via notebook (!pip install) only affects the current kernel")
    print("- Using uv in the terminal installs to your project's virtual environment")
    print("- This ensures consistency across all notebooks and scripts")
    print("\nAfter installing, restart the kernel to use the new package.")

## Understanding Ports and Network Communication

### Why Port Communication Matters

Port communication is foundational to modern computing for several key reasons:

1. **Enables Multi-Service Architecture**: Ports allow a single device to run multiple network services simultaneously
2. **Service Identification**: Helps identify which application or service should receive incoming data
3. **Security Boundary**: Provides a controlled interface for external systems to access specific services
4. **Network Troubleshooting**: Essential for diagnosing connectivity issues and network problems
5. **Application Development**: Critical knowledge for building distributed applications

### Real-World Applications of Port Communication

Port communication is essential in numerous domains:

- **Web Development**: HTTP/HTTPS servers, API gateways, microservices communication
- **IoT Systems**: Device communication, sensor data collection, remote control
- **Cloud Computing**: Load balancing, container orchestration, service discovery
- **Cybersecurity**: Network monitoring, intrusion detection, penetration testing
- **Game Development**: Multiplayer functionality, game servers, matchmaking services
- **Financial Systems**: Trading platforms, payment processing, banking systems
- **Industrial Control**: SCADA systems, industrial automation, remote monitoring

### What is a Port?

A port is like a numbered doorway into your computer:
- Each port number (0-65535) can be used for different services
- Common ports: 80 (HTTP), 443 (HTTPS), 22 (SSH), 3306 (MySQL)
- Multiple applications can listen on different ports
- Only one application can use a specific port at a time

### Common Port Ranges:
1. Well-known ports (0-1023)
   - Reserved for standard services
   - Require administrative privileges
   - Examples: HTTP (80), HTTPS (443), FTP (21)

2. Registered ports (1024-49151)
   - Used by applications
   - Don't require special privileges
   - Good for custom applications

3. Dynamic ports (49152-65535)
   - Used for temporary connections
   - Available for any purpose

Let's explore what ports are currently in use on your system:

In [ ]:
def list_active_connections():
    """List all active network connections and their ports"""
    connections = psutil.net_connections()
    
    print("Active Network Connections:")
    print("-" * 50)
    print(f"{'Local Address':<20} {'Local Port':<10} {'Status':<10}")
    print("-" * 50)
    
    for conn in connections:
        if conn.laddr:  # Only show connections with local addresses
            addr = conn.laddr.ip
            port = conn.laddr.port
            status = conn.status
            print(f"{addr:<20} {port:<10} {status:<10}")

try:
    list_active_connections()
except Exception as e:
    print(f"Error listing connections: {e}")
    print("Note: Some systems may require elevated privileges to list all connections.")


Let's create a basic TCP server that:
1. Listens on a specific port
2. Accepts connections from clients
3. Receives and sends messages
4. Handles multiple clients simultaneously

This example will help you understand:
- How servers listen for connections
- How clients connect to servers
- Basic socket programming concepts
- Error handling in network programming

In [ ]:
class EchoServer:
    def __init__(self, host='localhost', port=12345):
        self.host = host
        self.port = port
        self.server_socket = None
        self.running = False
        
    def start(self):
        """Start the server"""
        self.server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        
        # Allow port reuse (helpful for development)
        self.server_socket.setsockopt(socket.SOL_SOCKET, socket.SO_REUSEADDR, 1)
        
        try:
            self.server_socket.bind((self.host, self.port))
            self.server_socket.listen(5)
            self.running = True
            
            print(f"Server listening on {self.host}:{self.port}")
            
            # Start accepting connections in a separate thread
            accept_thread = threading.Thread(target=self._accept_connections)
            accept_thread.start()
            
        except Exception as e:
            print(f"Error starting server: {e}")
            self.stop()
    
    def _accept_connections(self):
        """Handle incoming connections"""
        while self.running:
            try:
                client_socket, address = self.server_socket.accept()
                print(f"New connection from {address}")
                
                # Handle each client in a separate thread
                client_thread = threading.Thread(
                    target=self._handle_client,
                    args=(client_socket, address)
                )
                client_thread.start()
                
            except Exception as e:
                if self.running:  # Only show error if server is still meant to be running
                    print(f"Error accepting connection: {e}")
    
    def _handle_client(self, client_socket, address):
        """Handle individual client connections"""
        try:
            while self.running:
                # Receive data from client
                data = client_socket.recv(1024).decode('utf-8')
                if not data:
                    break
                    
                print(f"Received from {address}: {data}")
                
                # Echo back to client
                response = f"Server received: {data}"
                client_socket.send(response.encode('utf-8'))
                
        except Exception as e:
            print(f"Error handling client {address}: {e}")
            
        finally:
            client_socket.close()
            print(f"Connection closed with {address}")
    
    def stop(self):
        """Stop the server"""
        self.running = False
        if self.server_socket:
            self.server_socket.close()
            print("Server stopped")

# Create and start the server
server = EchoServer()
server.start()

print("Server is running... (Stop kernel to stop the server)")

## Creating a TCP Client

Now let's create a client that can connect to our server. 

In this section, we'll build a simple client class with these capabilities:
- Establishing connections to servers
- Sending and receiving data
- Handling connection errors
- Properly cleaning up resources

Let's start by defining our client class and its initialization method:

In [ ]:
class EchoClient:
    def __init__(self, host='localhost', port=12345):
        """Initialize the client with server information"""
        self.host = host
        self.port = port
        self.socket = None
        
    def connect(self):
        """Connect to the server"""
        try:
            # Create a new TCP socket
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            
            # Connect to server using host and port
            self.socket.connect((self.host, self.port))
            
            print(f"Connected to server at {self.host}:{self.port}")
            return True
            
        except Exception as e:
            print(f"Error connecting to server: {e}")
            return False
    
    def send_message(self, message):
        """Send a message to the server and get the response"""
        # First check if we have an active connection
        if not self.socket:
            print("Not connected to server")
            return
        
        try:
            # Convert string message to bytes and send
            self.socket.send(message.encode('utf-8'))
            
            # Wait for and receive the response (up to 1024 bytes)
            response = self.socket.recv(1024).decode('utf-8')
            return response
            
        except Exception as e:
            print(f"Error sending/receiving message: {e}")
            return None
    
    def close(self):
        """Close the connection"""
        if self.socket:
            self.socket.close()
            self.socket = None
            print("Connection closed")

### The EchoClient Class Methods

The `EchoClient` class we created above includes three essential methods for network communication:

1. **`connect()`**: Establishes a connection to the server
   - Creates a new socket for communication
   - Attempts to connect to the specified host and port
   - Returns True on success or False on failure
   - Includes proper error handling for connection issues

2. **`send_message()`**: Sends data to the server and receives responses
   - Verifies an active connection exists before sending
   - Converts string messages to bytes for transmission
   - Receives and decodes response data from the server
   - Handles potential network errors during communication

3. **`close()`**: Properly cleans up the connection
   - Closes the socket to free system resources
   - Sets internal state to indicate disconnection
   - Prevents resource leaks from lingering connections

### Testing Our Client

Now let's test our client by:
1. Creating a client instance
2. Connecting to the server
3. Sending a couple of test messages
4. Receiving and displaying the responses
5. Properly closing the connection

In [ ]:
# Create a client instance
client = EchoClient()

# Try to connect to the server
if client.connect():
    # Test with first message
    response = client.send_message("Hello, Server!")
    print(f"Server response: {response}")
    
    # Test with second message
    response = client.send_message("How are you?")
    print(f"Server response: {response}")
    
    # Always close the connection when done
    client.close()
else:
    print("Could not connect to the server. Make sure it's running.")

## Creating an HTTP Server

While the echo server demonstrates basic port communication, let's create something more practical: a simple HTTP server.

HTTP servers are among the most common types of network services. Creating one will help you understand:
- How web servers work at a basic level
- The HTTP request/response cycle
- Handling different types of requests (GET, POST)
- Serving data over HTTP

Let's start by importing the required modules and creating a request handler class:

In [ ]:
# Import required modules
from http.server import HTTPServer, BaseHTTPRequestHandler
import json

### Creating the HTTP Request Handler

Now let's define our HTTP request handler class:

- **Inheritance**: Our `SimpleHTTPHandler` inherits from `BaseHTTPRequestHandler`
- **Response Helper**: A utility method to format and send JSON responses
- **HTTP Method Handlers**: Methods to handle different types of HTTP requests:
  - `do_GET` - For retrieving data (like viewing a webpage)
  - `do_POST` - For submitting data (like sending a form)

Let's implement this class with proper methods for each type of request:

In [ ]:
class SimpleHTTPHandler(BaseHTTPRequestHandler):
    def _send_response(self, message, status=200):
        """Helper method to send a response"""
        # Send HTTP status code (200 OK, 404 Not Found, etc.)
        self.send_response(status)
        
        # Set response headers
        self.send_header('Content-type', 'application/json')
        self.end_headers()
        
        # Convert message to JSON and encode as bytes
        response = json.dumps({"message": message}).encode('utf-8')
        
        # Write response body to output stream
        self.wfile.write(response)
    
    def do_GET(self):
        """Handle GET requests"""
        # Route based on the requested path
        if self.path == '/':
            # Root path - welcome message
            self._send_response("Welcome to the simple HTTP server!")
            
        elif self.path == '/time':
            # Current time endpoint
            current_time = time.strftime("%Y-%m-%d %H:%M:%S")
            self._send_response(f"Current time is: {current_time}")
            
        else:
            # Any other path - 404 Not Found
            self._send_response("Not found", 404)
    
    def do_POST(self):
        """Handle POST requests"""
        if self.path == '/echo':
            # Get the length of the incoming data
            content_length = int(self.headers['Content-Length'])
            
            # Read the posted data
            post_data = self.rfile.read(content_length)
            
            # Echo it back in the response
            self._send_response(f"Received: {post_data.decode('utf-8')}")
            
        else:
            # Any other path - 404 Not Found
            self._send_response("Not found", 404)

### HTTP Request Handlers

The `SimpleHTTPHandler` class now includes two important methods:

1. **`do_GET`**: Called when a client makes an HTTP GET request
   - Handles the root path `/` with a welcome message
   - Handles the `/time` path to return the current server time
   - Returns 404 for any other paths

2. **`do_POST`**: Called when a client makes an HTTP POST request
   - Handles the `/echo` path to echo back posted data
   - Returns 404 for any other paths

These methods are automatically called by the HTTP server framework based on the incoming request method.

### Running the HTTP Server

Now we need to create a function to start our HTTP server. This function will:

1. Create a server instance with our custom handler
2. Bind to a specified port (default 8000)
3. Handle serving requests continuously
4. Include clean shutdown logic

We'll run this server in a background thread so it doesn't block the notebook from further execution:

In [ ]:
def run_http_server(port=8000):
    """Run the HTTP server"""
    # Create server with empty host ('') to listen on all interfaces
    server_address = ('', port)
    httpd = HTTPServer(server_address, SimpleHTTPHandler)
    
    print(f"Starting HTTP server on port {port}...")
    
    try:
        # This will run until interrupted
        httpd.serve_forever()
    except KeyboardInterrupt:
        print("\nShutting down server...")
        httpd.server_close()

### Starting the Server in a Background Thread

Let's run our server in a background thread so it doesn't block the notebook.
This will allow us to continue using the notebook while the server is running:

In [ ]:
# Create and start a thread for the server
server_thread = threading.Thread(target=run_http_server)
server_thread.daemon = True  # Thread will close when notebook closes
server_thread.start()

print("HTTP Server is running on port 8000")
print("Try these URLs in your browser:")
print("  - http://localhost:8000/")
print("  - http://localhost:8000/time")

### Testing the HTTP Server

Now that our server is running, you can test it using various methods:

#### Browser Testing
Simply open your web browser and navigate to:
- http://localhost:8000/ - To see the welcome message
- http://localhost:8000/time - To get the current time
- http://localhost:8000/unknown - To see a 404 response

#### Command Line Testing with curl
You can test both GET and POST requests using curl:
```bash
# Test GET request
curl http://localhost:8000/

# Test POST request
curl -X POST -d "Hello from curl!" http://localhost:8000/echo
```

#### Testing with Python requests
You can also test from another notebook cell or Python script:
```python
import requests

# Test GET request
response = requests.get('http://localhost:8000/')
print(response.json())

# Test POST request
response = requests.post('http://localhost:8000/echo', data='Hello from Python!')
print(response.json())
```

If you see "501 - Server does not support this operation" error, make sure you've run all the cells in order, particularly the cell containing the complete `SimpleHTTPHandler` class with both `do_GET` and `do_POST` methods properly defined.